In [1]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets.widgets import Layout, HBox, VBox
import bqplot as bqp 
import bqplot.pyplot as plt

In [2]:
data_df = pd.read_csv('data.csv')
data_df = data_df[~data_df["YearFrom"].isin(["16..","17.."])]  
data_df = data_df.astype({"YearFrom": float})

In [3]:
# pd.options.display.max_columns = None
# data_df

In [4]:
year_range=widgets.IntRangeSlider(
    value=[1500, 1803],
    min=1500, max=1803, step=1,
    description='Select Year:',
    layout=Layout(width='300px'),
    continuous_update=False)

years_filter = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to unselect',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

age_filter = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to unselect',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

gender_filter = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to unselect',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

profession_filter = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to unselect',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

crime_filter = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to unselect',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

punishment_filter = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to unselect',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

In [5]:
crime_df =data_df["Misdrijven/Crimes"].value_counts().rename_axis('unique_values').reset_index(name='counts')
crime_df = crime_df.sort_values('counts', ascending=False).head(5)
punishment_df =data_df['punishment(fand)/Punishmandts/Pandalties'].value_counts().rename_axis('unique_values').reset_index(name='counts')
punishment_df = punishment_df.sort_values('counts', ascending=False).head(5)

trend_df = data_df["YearFrom"].value_counts().rename_axis('unique_values').reset_index(name='counts')

age_df = data_df.loc[data_df['Age'].notnull(), ['Age']]
age_df.index = pd.RangeIndex(len(age_df.index))

gender_df = data_df["VerdachteGeslacht/AccusedGandder"].value_counts().rename_axis('unique_values').reset_index(name='counts')

profession_df = data_df["Suspect Professional/AccusedProfession"].value_counts().rename_axis('unique_values').reset_index(name='counts')
profession_df = profession_df.sort_values('counts', ascending=False).head(5)

clicked_year_range = []
clicked_age_range = []
clicked_gender = None
clicked_profession = None
clicked_crime = None
clicked_punishment = None

In [6]:
trend_x_sc = bqp.LinearScale()
trend_y_sc = bqp.LinearScale()

trend_x_ax = bqp.Axis(
    label='Year', 
    scale=trend_x_sc, 
    grid_lines='solid', 
)

trend_y_ax = bqp.Axis(
    label='Number of Crimes', 
    scale=trend_y_sc, 
    orientation='vertical', 
)

trend = bqp.Scatter(x=trend_df['unique_values'], y=trend_df['counts'], scales={'x': trend_x_sc, 'y': trend_y_sc} , colors=['blue'],display_legend = False, labels=['y1'],stroke_width = 1.0) 
trend_fig = bqp.Figure(
    layout=Layout(width='300px'),
    axes=[trend_x_ax, trend_y_ax], 
    marks=[trend], 
    animation_duration=500)

gender_x_ord = bqp.OrdinalScale()
gender_y_sc = bqp.LinearScale()

gender_bar = bqp.Bars(
    x = gender_df['unique_values'],
    y = gender_df['counts'],
    interactions={'click': 'select'},
    scales={'x': gender_x_ord, 'y': gender_y_sc}, 
    orientation='horizontal'
)

gender_x_ax = bqp.Axis(
    scale=gender_y_sc
)

gender_y_ax = bqp.Axis(
    scale=gender_x_ord,
    orientation='vertical'
)

gender_fig = bqp.Figure(
    axes=[gender_x_ax, gender_y_ax],
    marks=[gender_bar],
    animation_duration=500
)

age_x_sc = bqp.LinearScale()
age_y_sc = bqp.LinearScale()

age_hist = bqp.Hist(
    sample = age_df['Age'],
    interactions={'click': 'select'},
    scales={'sample': age_x_sc, 'count': age_y_sc}, 
)

age_x_ax = bqp.Axis(
    scale=age_x_sc
)

age_y_ax = bqp.Axis(
    scale=age_y_sc,
    orientation='vertical'
)

age_fig = bqp.Figure(
    axes=[age_x_ax, age_y_ax],
    marks=[age_hist],
    animation_duration=500
)

profession_x_ord = bqp.OrdinalScale()
profession_y_sc = bqp.LinearScale()

profession_x_ax = bqp.Axis(
    scale=profession_y_sc
)

profession_y_ax = bqp.Axis(
    scale=profession_x_ord,
    orientation='vertical'
)


profession_bar = bqp.Bars(
    x = profession_df['unique_values'],
    y = profession_df['counts'],
    interactions={'click': 'select'},
    scales={'x': profession_x_ord, 'y': profession_y_sc}, 
    orientation='horizontal'
)

profession_fig = bqp.Figure(
    axes=[profession_x_ax, profession_y_ax],
    marks=[profession_bar],
    animation_duration=500
)

crime_x_ord = bqp.OrdinalScale()
crime_y_sc = bqp.LinearScale()

crime_bar = bqp.Bars(
    x = crime_df['unique_values'],
    y = crime_df['counts'],
    interactions={'click': 'select'},
    scales={'x': crime_x_ord, 'y': crime_y_sc}, 
    orientation='horizontal'
)

crime_x_ax = bqp.Axis(
    scale=crime_y_sc
)

crime_y_ax = bqp.Axis(
    scale=crime_x_ord,
    orientation='vertical'
)

crime_fig = bqp.Figure(
    axes=[crime_x_ax, crime_y_ax],
    marks=[crime_bar],
    animation_duration=500
)

punishment_x_ord = bqp.OrdinalScale()
punishment_y_sc = bqp.LinearScale()

punishment_x_ax = bqp.Axis(
    scale=punishment_y_sc
)

punishment_y_ax = bqp.Axis(
    scale=punishment_x_ord,
    orientation='vertical'
)


punishment_bar = bqp.Bars(
    x = punishment_df['unique_values'],
    y = punishment_df['counts'],
    interactions={'click': 'select'},
    scales={'x': punishment_x_ord, 'y': punishment_y_sc}, 
    orientation='horizontal'
)

punishment_fig = bqp.Figure(
    axes=[punishment_x_ax, punishment_y_ax],
    marks=[punishment_bar],
    animation_duration=500
)

In [7]:
def filter_df_and_update_plots(age_flag=True, gender_flag=True, profession_flag=True, crime_flag=True, punishment_flag=True):
    filtered_df = data_df
    if clicked_year_range != []:
        if clicked_year_range[0]==clicked_year_range[1]:
            filtered_df = filtered_df.loc[filtered_df["YearFrom"]==clicked_year_range[0]]
        else:
            filtered_df = filtered_df.loc[filtered_df["YearFrom"].isin(range(clicked_year_range[0], clicked_year_range[1], 1))]
    if clicked_age_range != []:
        if clicked_age_range[0]==clicked_age_range[1]:
            filtered_df = filtered_df.loc[filtered_df["Age"]==clicked_age_range[0]]
        else:
            filtered_df = filtered_df.loc[filtered_df["Age"].isin(range(int(clicked_age_range[0]), int(clicked_age_range[1]), 1))]
    if not(clicked_gender is None):
        filtered_df = filtered_df.loc[filtered_df["VerdachteGeslacht/AccusedGandder"]==clicked_gender]
    if not(clicked_profession is None):
        filtered_df = filtered_df.loc[filtered_df["VerdachteGeslacht/AccusedGandder"]==clicked_profession]
    if not(clicked_crime is None):
        filtered_df = filtered_df.loc[filtered_df["VerdachteGeslacht/AccusedGandder"]==clicked_crime]
    if not(clicked_punishment is None):
        filtered_df = filtered_df.loc[filtered_df["VerdachteGeslacht/AccusedGandder"]==clicked_punishment]
    
    if age_flag:
        age_df = filtered_df.loc[filtered_df['Age'].notnull(), ['Age']]
        age_df.index = pd.RangeIndex(len(age_df.index))
        age_hist.sample=age_df['Age']
    
    if gender_flag:
        gender_df = filtered_df["VerdachteGeslacht/AccusedGandder"].value_counts().rename_axis('unique_values').reset_index(name='counts')
        gender_bar.x = gender_df['unique_values'],
        gender_bar.y = gender_df['counts'],
   
    if profession_flag:
        profession_df = filtered_df["Suspect Professional/AccusedProfession"].value_counts().rename_axis('unique_values').reset_index(name='counts')
        profession_df = profession_df.sort_values('counts', ascending=False).head(5)
        profession_bar.x = profession_df['unique_values'],
        profession_bar.y = profession_df['counts'],
    
    if crime_flag:
        crime_df =filtered_df["Misdrijven/Crimes"].value_counts().rename_axis('unique_values').reset_index(name='counts')
        crime_df = crime_df.sort_values('counts', ascending=False).head(5)
        crime_bar.x = crime_df['unique_values'],
        crime_bar.y = crime_df['counts'],
    
    if punishment_flag:
        punishment_df =filtered_df['punishment(fand)/Punishmandts/Pandalties'].value_counts().rename_axis('unique_values').reset_index(name='counts')
        punishment_df = punishment_df.sort_values('counts', ascending=False).head(5)
        punishment_bar.x = punishment_df['unique_values'],
        punishment_bar.y = punishment_df['counts'],
    
def refresh_filters(flag=True, age_flag=True, gender_flag=True, profession_flag=True, crime_flag=True, punishment_flag=True):
    if flag:
        filter_df_and_update_plots(age_flag, gender_flag, profession_flag, crime_flag, punishment_flag)
    filter_list = [years_filter, age_filter, gender_filter, profession_filter, crime_filter, punishment_filter]
    filter_box.children =(widgets.Label(value="Selected filters, click to unselect"),)+tuple([f for f in filter_list if f.description != ""])

def select_years(y):
    global clicked_year_range
    if year_range.value[0] == year_range.min and year_range.value[1] == year_range.max:
        clicked_year_range = []
        years_filter.description=''
        refresh_filters(flag=False)
    else:
        clicked_year_range = year_range.value
        years_filter.description=str(clicked_year_range[0])+'~'+str(clicked_year_range[1])
        refresh_filters()
        trend_x_sc.min = clicked_year_range[0]
        trend_x_sc.max = clicked_year_range[1]

def click_age_hist(clicked):
    global clicked_age_range
    if clicked['new'] is None or len(clicked['new']) == 0:
        pass
    else:
        age_series = age_df[age_df.index.isin(clicked['new'])]['Age']
        clicked_age_range = [age_series.min(), age_series.max()]
        age_filter.description='Age'+str(clicked_age_range[0])+'~'+str(clicked_age_range[1])
        refresh_filters(age_flag=False)
    
def click_gender_bar(clicked):
    global clicked_gender 
    clicked_gender = gender_df['unique_values'][clicked['new'][0]]
    gender_filter.description=clicked_gender
    refresh_filters(age_flag=False,gender_flag=False)
    
def click_profession_bar(clicked):
    global clicked_profession
    clicked_profession = profession_df['unique_values'][clicked['new'][0]]
    profession_filter.description=clicked_profession
    refresh_filters(age_flag=False,gender_flag=False,profession_flag=False)
    
def click_crime_bar(clicked):
    global clicked_crime
    clicked_crime = crime_df['unique_values'][clicked['new'][0]]
    crime_filter.description=clicked_crime
    refresh_filters(age_flag=False,gender_flag=False,profession_flag=False,crime_flag=False)
    
def click_punishment_bar(clicked):
    global clicked_punishment
    clicked_punishment = punishment_df['unique_values'][clicked['new'][0]]
    punishment_filter.description= clicked_punishment
    refresh_filters(age_flag=False,gender_flag=False,profession_flag=False,crime_flag=False,punishment_flag=False)

def click_years_filter(clicked):
    global clicked_year_range
    clicked_year_range = []
    years_filter.description=''
    refresh_filters()
    
def click_age_filter(clicked):
    global clicked_age_range 
    clicked_age_range = []
    age_filter.description=''
    refresh_filters()
    
def click_gender_filter(clicked):
    global clicked_gender
    clicked_gender = None
    gender_filter.description=''
    refresh_filters()
    
def click_profession_filter(clicked):
    global clicked_profession
    clicked_profession = None
    profession_filter.description=''
    refresh_filters()
    
def click_crime_filter(clicked):
    global clicked_crime
    clicked_crime = None
    crime_filter.description=''
    refresh_filters()
    
def click_punishment_filter(clicked):
    global clicked_punishment
    clicked_punishment = None
    punishment_filter.description=''
    refresh_filters()

In [8]:
#assign widget actions to functions
year_range.observe(select_years, names = 'value')
age_hist.observe(click_age_hist, 'selected')
gender_bar.observe(click_gender_bar, 'selected')
profession_bar.observe(click_profession_bar, 'selected')
crime_bar.observe(click_crime_bar, 'selected')
punishment_bar.observe(click_punishment_bar, 'selected')

years_filter.on_click(click_years_filter)
age_filter.on_click(click_age_filter)
gender_filter.on_click(click_gender_filter)
profession_filter.on_click(click_profession_filter)
crime_filter.on_click(click_crime_filter)
punishment_filter.on_click(click_punishment_filter)

In [9]:
#display all
filter_box = HBox(
    children=(widgets.Label(value="Selected filters, click to unselect"),),
)

top_right_box = VBox(
    children=(trend_fig, year_range),
)

top_box = HBox(
    children=(age_fig,top_right_box),
)

bottom_left_box = VBox(
    children=(gender_fig, profession_fig),
)

bottom_right_box = VBox(
    children=(crime_fig, punishment_fig),
)

bottom_box = HBox(
    children=(bottom_left_box, bottom_right_box),
)

vert_box  = VBox(
    children=(filter_box, top_box, bottom_box),
)

display (vert_box)

ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set

ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set

ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set

ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set